### Import cell

In [299]:

import os
import sys
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import (load_nerl_data,
    get_normalized_adj,
    get_Laplace,
    calculate_random_walk_matrix,
    test_error_cap,
    test_error
)
import random
import copy
import scipy
from scipy.io import loadmat
import math
import pandas as pd
from basic_structure import D_GCN, C_GCN, K_GCN,IGNNK
from basic_process import *
from sklearn.manifold import TSNE
import os
import pandas as pd
from sklearn import linear_model
from pandas import read_csv 

In [300]:
from utils import *

In [315]:
n_o_n_m = 8 #sampled space dimension

h = 24 #sampled time dimension

z = 100 #hidden dimension for graph convolution

K = 1 #If using diffusion convolution, the actual diffusion convolution step is K+1

n_m = 5 #number of mask node during training

N_u = 5 #target locations, N_u locations will be deleted from the training data

Max_episode = 750 #max training episode

learning_rate = 0.0001 #the learning_rate for Adam optimizer

batch_size = 8

E_maxvalue=80

STmodel = IGNNK(h, z, K)
# STmodel.load_state_dict(torch.load('model/IGNNK_ushcn_750iter_2023-11-26 12:25:53.pth'))

## Read Data

## Shenzhen Data

In [86]:
# def get_all_filenames(folder_path):
#     filenames = []
#     for filename in os.listdir(folder_path):
#         filenames.append(filename)
#     return filenames

In [87]:
# folder_path = "D:\Spatial_interpolation\SSIN\data\Air_quality-Shenzhen"
# all_filenames = get_all_filenames(folder_path)

In [88]:
# file_result=[]
# for filename in all_filenames:
#     x=pd.read_csv(folder_path+"/"+filename)
#     x=np.array(x)
#     x=np.reshape(x,(7296, 6 ,1))
#     file_result.append(x)

In [89]:
# result=file_result[0]

In [90]:
# for i in range(1,len(file_result)):
#     result = np.append(result,file_result[i] , axis=2)

In [91]:
# result.shape
# X_raw=result

In [ ]:
# Station_info = pd.read_csv("D:\Spatial_interpolation\SSIN\data\location_of_Shenzhen.csv")

## NDBC data

In [198]:
# #Read all data
# pathroot='data/NDBC/all_stations'
# PATH_ROOT = os.getcwd()
# ROOT = os.path.join(PATH_ROOT, pathroot)
# filenames = os.listdir(ROOT)
# # Sort all files
# filenames.sort()
# data = []
# for i in filenames:
#     PATH_CSV = os.path.join(ROOT, i)
#     with open(PATH_CSV, 'r') as file:
# # Use splitlines() to divide contents in the documents into lists.
#         content_list = file.read().splitlines()
# # Transform lists into Numpy.
#     content_matrix = np.array([list(map(float, line.split())) for line in content_list])
#     data.append(content_matrix)
# data = np.array(data).transpose(1, 2, 0)
# X_raw=data

In [199]:
# Station_info=pd.read_csv('data/NDBC/Station_info.csv')

## Te_hiku Data

In [302]:
X_raw=np.load('D:\Spatial_interpolation\SSIN\data\Te_Hiku/all.npy',allow_pickle=True)
X_raw=np.array(X_raw)

In [303]:
Station_info = pd.read_csv("D:\Spatial_interpolation\SSIN\data\Te_Hiku/Tehiku_coordinate.csv")
Station_info=Station_info.T

In [304]:
Station_info=Station_info.iloc[1:,1:]
Station_info = Station_info.rename(columns={1: 'lon', 2: 'lat'})

In [305]:
X_raw.shape

(34141, 7, 24)

### Experiment 1: Raw setting on IGNNK without training

In [6]:
for j in range(5,X_raw.shape[1]-1):
        NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
        NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
        NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
        Adj_dist=adj_dist(NDBC_lat,NDBC_long)
        X_raw_0=X_raw[:,j,:]#GET the first feature
        #print(X_raw_0.shape)
        # Count the num of missing values in each column
        missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
        # print results
       # print("Incomplete data number in each column：", missing_counts_per_column)
        #Get the index if the value is not zero
        # Find the columns where missing values exist.
        columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

        # Get the column numbers when missing value exist.
        missing_columns = np.where(columns_with_missing_data)[0]

        #Get new data after deletion.
        #Delete those columns(Stations) if there is not any features.
        result = np.delete(X_raw_0, missing_columns, axis=1)
        result = (result - np.min(result)) / (np.max(result) - np.min(result))
        #Mean results---for SNE
        mean_result_axis_1=np.mean(result,axis=0)
        mean_result_axis_1=mean_result_axis_1.reshape(-1,1)        #Get new adjacency matrix after deletion.
        NDBC_long=NDBC_long.transpose()
        NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
        # Reconstruct the index in the DataFrame and get the longitude after deletion.
        NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
        Temp_long=NDBC_long_aft.values

        NDBC_lat=NDBC_lat.transpose()
        NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
        # Reconstruct the index in the DataFrame and get the latitude after deletion.
        NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
        Temp_lat=NDBC_lat_aft.values

        merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
        tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
        array_aft = tsne.fit_transform(merged_array)

        #GEt the new ADJ matrix.
        Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
        capacities=np.ones((Adj_dist.shape[0], 1))
        capacities=capacities.flatten()
        split_line1 = int(result.shape[0] * 0.7)
        training_set = result[:split_line1, :]
        test_set = result[split_line1:, :]
        rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
        unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
        unknow_set = set(unknow_set)
        full_set = set(range(0,result.shape[1]))
        know_set = full_set - unknow_set
        training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
        A_s = Adj_dist[:, list(know_set)][list(know_set), :]
        MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
        print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

Best model in the 5 -th column is 0.5117526179304062 0.6237494276278088 -3.5100974172557624
Best model in the 6 -th column is 0.22314292522461132 0.2802671951588756 -6.252086316264561
Best model in the 7 -th column is 0.1978166269787692 0.24901721940629162 -7.754746218833569
Best model in the 8 -th column is 0.17461698724298205 0.2138096613489656 -6.752101662992679
Best model in the 9 -th column is 0.31846986084755563 0.3912929218861143 -5.665866334182303
Best model in the 10 -th column is 0.20591907538099286 0.25953467957330045 -9.818228978758263
Best model in the 11 -th column is 0.5000757252849869 0.5972434135667379 -3.3326970731051153
Best model in the 12 -th column is 0.15358280238806749 0.17983403867889008 -211.96156576706565
Best model in the 13 -th column is 0.76474415030283 0.8627515015882451 -67.4299460656589
Best model in the 14 -th column is 0.5617500868539255 0.6552130422354571 -8.162254964090867
Best model in the 15 -th column is 0.6718070330144668 0.7196767516778344 -20.

In [176]:
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,0])
# NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
X_raw_0=X_raw[:,2,:]#GET the first feature

In [158]:

X_raw_0 = X_raw_0.astype(float)

In [180]:
missing_columns

array([ 2, 10, 20], dtype=int64)

In [183]:
NDBC_long

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P15,P16,P17,P18,P20,P21,P22,P23,P24,P25
lon,173.10635,173.1264,173.142486,173.115715,173.13217,173.118677,173.1489,173.108604,173.146267,173.14545,...,173.113154,173.142486,173.113248,173.118065,173.119353,173.12386,173.132426,173.13378,173.122335,173.116352


## For 1 feature -- train

### Expect te_hiku

In [312]:
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
X_raw_0=X_raw[:,2,:]#GET the first feature
# X_raw_0 = X_raw_0.astype(float)
#print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
# print("Incomplete data number in each column：", missing_counts_per_column)
#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

#Get new data after deletion.
#Delete those columns(Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
        #Mean results---for SNE
mean_result_axis_1=np.mean(result,axis=0)
mean_result_axis_1=mean_result_axis_1.reshape(-1,1)        #Get new adjacency matrix after deletion.
NDBC_long=NDBC_long.transpose()
NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the longitude after deletion.
NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
Temp_long=NDBC_long_aft.values

NDBC_lat=NDBC_lat.transpose()
NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the latitude after deletion.
NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
Temp_lat=NDBC_lat_aft.values

merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
array_aft = tsne.fit_transform(merged_array)

#GEt the new ADJ matrix.
Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
capacities=np.ones((Adj_dist.shape[0], 1))
capacities=capacities.flatten()
split_line1 = int(result.shape[0] * 0.7)
training_set = result[:split_line1, :]
test_set = result[split_line1:, :]
rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
unknow_set = set(unknow_set)
full_set = set(range(0,result.shape[1]))
know_set = full_set - unknow_set
training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
A_s = Adj_dist[:, list(know_set)][list(know_set), :]

IndexError: single positional indexer is out-of-bounds

### Te_hiku

In [316]:
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,0])
# NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
X_raw_0=X_raw[:,6,:]#GET the first feature
X_raw_0 = X_raw_0.astype(float)
#print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
# print("Incomplete data number in each column：", missing_counts_per_column)
#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

#Get new data after deletion.
#Delete those columns(Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
        #Mean results---for SNE
mean_result_axis_1=np.mean(result,axis=0)
mean_result_axis_1=mean_result_axis_1.reshape(-1,1)        #Get new adjacency matrix after deletion.
NDBC_long=NDBC_long.transpose()
NDBC_long=NDBC_long.rename(columns={"P1": 0,"P2": 1,"P3": 2,"P4": 3,"P5": 4,"P6": 5,"P7": 6,"P8": 7,"P9": 8,"P10": 9,"P11": 10,"P12": 11,"P13": 12,"P14": 13,"P14": 13,"P15": 14,"P16": 15,"P17": 16,"P18": 17,"P19": 18,"P20": 19,"P21": 20,"P22": 21,"P23": 22,"P24": 23,"P25": 24})

NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the longitude after deletion.
NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
Temp_long=NDBC_long_aft.values

NDBC_lat=NDBC_lat.transpose()
NDBC_lat=NDBC_lat.rename(columns={"P1": 0,"P2": 1,"P3": 2,"P4": 3,"P5": 4,"P6": 5,"P7": 6,"P8": 7,"P9": 8,"P10": 9,"P11": 10,"P12": 11,"P13": 12,"P14": 13,"P14": 13,"P15": 14,"P16": 15,"P17": 16,"P18": 17,"P19": 18,"P20": 19,"P21": 20,"P22": 21,"P23": 22,"P24": 23,"P25": 24})

NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the latitude after deletion.
NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
Temp_lat=NDBC_lat_aft.values

merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
array_aft = tsne.fit_transform(merged_array)

#GEt the new ADJ matrix.
Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
capacities=np.ones((Adj_dist.shape[0], 1))
capacities=capacities.flatten()
split_line1 = int(result.shape[0] * 0.7)
training_set = result[:split_line1, :]
test_set = result[split_line1:, :]
rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
unknow_set = set(unknow_set)
full_set = set(range(0,result.shape[1]))
know_set = full_set - unknow_set
training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
A_s = Adj_dist[:, list(know_set)][list(know_set), :]

In [317]:
# NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
# NDBC_long=pd.DataFrame(Station_info.iloc[:,2])
# NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
# Adj_dist=adj_dist(NDBC_lat,NDBC_long)
# X_raw_0=X_raw[:,5,:]#GET the first feature
# #print(X_raw_0.shape)
# # Count the num of missing values in each column
# missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# # print results
# # print("Incomplete data number in each column：", missing_counts_per_column)
# #Get the index if the value is not zero
# # Find the columns where missing values exist.
# columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# # Get the column numbers when missing value exist.
# missing_columns = np.where(columns_with_missing_data)[0]

# #Get new data after deletion.
# #Delete those columns(Stations) if there is not any features.
# result = np.delete(X_raw_0, missing_columns, axis=1)
# result = (result - np.min(result)) / (np.max(result) - np.min(result))
#         #Mean results---for SNE
# mean_result_axis_1=np.mean(result,axis=0)
# mean_result_axis_1=mean_result_axis_1.reshape(-1,1)        #Get new adjacency matrix after deletion.
# NDBC_long=NDBC_long.transpose()
# NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
# # Reconstruct the index in the DataFrame and get the longitude after deletion.
# NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
# Temp_long=NDBC_long_aft.values

# NDBC_lat=NDBC_lat.transpose()
# NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
# # Reconstruct the index in the DataFrame and get the latitude after deletion.
# NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
# Temp_lat=NDBC_lat_aft.values

# merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
# tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
# array_aft = tsne.fit_transform(merged_array)

# #GEt the new ADJ matrix.
# Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
# capacities=np.ones((Adj_dist.shape[0], 1))
# capacities=capacities.flatten()
# split_line1 = int(result.shape[0] * 0.7)
# training_set = result[:split_line1, :]
# test_set = result[split_line1:, :]
# rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
# unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
# unknow_set = set(unknow_set)
# full_set = set(range(0,result.shape[1]))
# know_set = full_set - unknow_set
# training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
# A_s = Adj_dist[:, list(know_set)][list(know_set), :]
# MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
# print('Best model in the',5, '-th column is', MAE_t, RMSE_t, R2_t)
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(Max_episode):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                # For other datasets, it is not necessary to mask 0 values
                                                
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
            
        Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        
        outputs = torch.from_numpy((inputs/E_maxvalue).astype('float32')) #The label
        
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    
    MAE_t, RMSE_t, MAPE_t, metr_ignnk_res  = test_error(STmodel, unknow_set, test_set, Adj_dist,E_maxvalue, True)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    if epoch%50 == 0:
        print(epoch, MAE_t, RMSE_t, MAPE_t)
idx = MAE_list == min(MAE_list)
print('Best model result:',np.array(MAE_list)[idx],np.array(RMSE_list)[idx],np.array(MAPE_list)[idx])

0 1.180307916337216 1.6101589753660124 -71.07424484962729
50 0.3053239619905196 0.3441877150615952 -2.293315337554632


KeyboardInterrupt: 

### Test for 1 feature

### Except Te_hiku

In [116]:
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,2])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
X_raw_0=X_raw[:,5,:]#GET the first feature
#print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
# print("Incomplete data number in each column：", missing_counts_per_column)
#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

#Get new data after deletion.
#Delete those columns(Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
        #Mean results---for SNE
mean_result_axis_1=np.mean(result,axis=0)
mean_result_axis_1=mean_result_axis_1.reshape(-1,1)        #Get new adjacency matrix after deletion.
NDBC_long=NDBC_long.transpose()
NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the longitude after deletion.
NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
Temp_long=NDBC_long_aft.values

NDBC_lat=NDBC_lat.transpose()
NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the latitude after deletion.
NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
Temp_lat=NDBC_lat_aft.values

merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
array_aft = tsne.fit_transform(merged_array)

#GEt the new ADJ matrix.
Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
capacities=np.ones((Adj_dist.shape[0], 1))
capacities=capacities.flatten()
split_line1 = int(result.shape[0] * 0.7)
training_set = result[:split_line1, :]
test_set = result[split_line1:, :]
rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
unknow_set = set(unknow_set)
full_set = set(range(0,result.shape[1]))
know_set = full_set - unknow_set
training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
A_s = Adj_dist[:, list(know_set)][list(know_set), :]
MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
print('Best model in the',6, '-th column is', MAE_t, RMSE_t, R2_t)

Best model in the 6 -th column is 0.013687401791334364 0.018425529628227567 -0.9039866963957004


### Te_hiku

In [298]:
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,0])
# NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
X_raw_0=X_raw[:,6,:]#GET the first feature
X_raw_0 = X_raw_0.astype(float)
#print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
# print("Incomplete data number in each column：", missing_counts_per_column)
#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

#Get new data after deletion.
#Delete those columns(Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
        #Mean results---for SNE
mean_result_axis_1=np.mean(result,axis=0)
mean_result_axis_1=mean_result_axis_1.reshape(-1,1)        #Get new adjacency matrix after deletion.
NDBC_long=NDBC_long.transpose()
NDBC_long=NDBC_long.rename(columns={"P1": 0,"P2": 1,"P3": 2,"P4": 3,"P5": 4,"P6": 5,"P7": 6,"P8": 7,"P9": 8,"P10": 9,"P11": 10,"P12": 11,"P13": 12,"P14": 13,"P14": 13,"P15": 14,"P16": 15,"P17": 16,"P18": 17,"P19": 18,"P20": 19,"P21": 20,"P22": 21,"P23": 22,"P24": 23,"P25": 24})

NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the longitude after deletion.
NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
Temp_long=NDBC_long_aft.values

NDBC_lat=NDBC_lat.transpose()
NDBC_lat=NDBC_lat.rename(columns={"P1": 0,"P2": 1,"P3": 2,"P4": 3,"P5": 4,"P6": 5,"P7": 6,"P8": 7,"P9": 8,"P10": 9,"P11": 10,"P12": 11,"P13": 12,"P14": 13,"P14": 13,"P15": 14,"P16": 15,"P17": 16,"P18": 17,"P19": 18,"P20": 19,"P21": 20,"P22": 21,"P23": 22,"P24": 23,"P25": 24})

NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the latitude after deletion.
NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
Temp_lat=NDBC_lat_aft.values

merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
array_aft = tsne.fit_transform(merged_array)

#GEt the new ADJ matrix.
Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
capacities=np.ones((Adj_dist.shape[0], 1))
capacities=capacities.flatten()
split_line1 = int(result.shape[0] * 0.7)
training_set = result[:split_line1, :]
test_set = result[split_line1:, :]
rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
unknow_set = set(unknow_set)
full_set = set(range(0,result.shape[1]))
know_set = full_set - unknow_set
training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
A_s = Adj_dist[:, list(know_set)][list(know_set), :]
MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
print('Best model in the',4, '-th column is', MAE_t, RMSE_t, R2_t)

Best model in the 4 -th column is 0.30709192214238734 0.4933594673092865 -6.2200239569607705


### Experiment 3.1: IGNNK-Training-dis-MI-based graph-3DGNN (Format:Metd-Adj-Only one defined graph-3DGNN)

In [7]:
def shan_entropy(c):
    import numpy as np
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))
    return H
def calc_MI(X,Y,bins):
    import numpy as np
    c_XY = np.histogram2d(X,Y,bins)[0]
    c_X = np.histogram(X,bins)[0]
    c_Y = np.histogram(Y,bins)[0]
    H_X = shan_entropy(c_X)
    H_Y = shan_entropy(c_Y)
    H_XY = shan_entropy(c_XY)

    MI = H_X + H_Y - H_XY
    return MI

In [8]:
#Subgraph generation module 1 --->Giving raw series, and generate a series of subgraph based on MI.
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
j=5#J-th colum, start from 5 as the first feature.
X_raw_0=X_raw[:,j,:]#GET the first feature
print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
print("Incomplete data number in each column：", missing_counts_per_column)

#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

# Print results
print("The column numbers with missing values are：", missing_columns)

# Get new data after deletion.
# Delete those columns (Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
print(result.shape)#(8784, 93)

#Determine group num here
group_num=result.shape[1]-N_u

# Initialize MI_store
MI_store = np.zeros((result.shape[1], result.shape[1]))

# Calculate mutual information
for i in range(result.shape[1]):
    for j in range(result.shape[1]):
        MI_store[i, j] = calc_MI(result[:, i], result[:, j], 10)

# Return column numbers and corresponding mutual information
column_numbers = []
MI_values = []

for i in range(MI_store.shape[0]):
    row = MI_store[i, :]
    row_index = np.arange(MI_store.shape[0])
    row_index = np.delete(row_index, i)  # Exclude the current row's index
    sorted_indices = np.argsort(row[row_index])
    max_indices = row_index[sorted_indices[-group_num:][::-1]]  # Get largest column_num
    column_numbers.append(max_indices)
    MI_values.append([row[j] for j in max_indices])

# Output results
for i, (columns, values) in enumerate(zip(column_numbers, MI_values)):
    print(f"Row {i} - Column Numbers: {columns}, MI Values: {values}")

(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175463517, 0.37428008133733837, 0.37263024544485734, 0.3321

In [12]:
def load_ndbc_data():
    pathroot='D:\Spatial_interpolation\IGNNK\data\NDBC\all_stations'
    PATH_ROOT = os.getcwd()
    ROOT = os.path.join(PATH_ROOT, pathroot)
    filenames = os.listdir(ROOT)
    # Sort all files
    filenames.sort()
    data = []
    for i in filenames:
        PATH_CSV = os.path.join(ROOT, i)
        with open(PATH_CSV, 'r') as file:
    # Use splitlines() to divide contents in the documents into lists.
            content_list = file.read().splitlines()
    # Transform lists into Numpy.
        content_matrix = np.array([list(map(float, line.split())) for line in content_list])
        data.append(content_matrix)
    data = np.array(data).transpose(1, 2, 0)
    X_raw=data
    # Raw data and raw adjacency matrix without mapminmax
    Station_info = pd.read_csv('D:\Spatial_interpolation\IGNNK\data\NDBC\Station_info.csv')
    NDBC_lat = pd.DataFrame(Station_info.iloc[:, 1])
    NDBC_long = pd.DataFrame(Station_info.iloc[:, 3])
    NDBC_ID = pd.DataFrame(Station_info.iloc[:, 0])
    Adj_dist = adj_dist(NDBC_lat, NDBC_long)
    j = 5  # J-th colum, start from 5 as the first feature.
    X_raw_0 = X_raw[:, j, :]  # GET the first feature
    # print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    # print("Incomplete data number in each column：", missing_counts_per_column)

    # Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    # print("The column numbers with missing values are：", missing_columns)

    # Get new data after deletion.
    # Delete those columns(Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    # Get new adjacency matrix after deletion.
    NDBC_long = NDBC_long.transpose()
    NDBC_long_aft = NDBC_long.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the longitude after deletion.
    NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
    print(NDBC_long_aft.shape)

    NDBC_lat = NDBC_lat.transpose()
    NDBC_lat_aft = NDBC_lat.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the latitude after deletion.
    NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
    # print(NDBC_lat_aft.shape)
    # GEt the new ADJ matrix.
    Adj_dist = adj_dist(NDBC_lat_aft.transpose(), NDBC_long_aft.transpose())
    X=result.transpose()
    A=Adj_dist
    X = X.astype(np.float32)
    return A, X

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 35-36: malformed \N character escape (249087588.py, line 2)

In [8]:
def load_data_MI(dataset):
    '''Load dataset
    Input: dataset name
    Returns
    -------
    A: adjacency matrix
    X: processed data
    capacity: only works for NREL, each station's capacity
    '''
    capacity = []
    if dataset == 'metr':
        A, X = load_metr_la_rdata()
        X = X[:,0,:]
    elif dataset == 'nrel':
        A, X , files_info = load_nerl_data()
        #For Nrel, We only use 7:00am to 7:00pm as the target data, because otherwise the 0-values of periods without sunshine will greatly influence the results
        time_used_base = np.arange(84,228)
        time_used = np.array([])
        for i in range(365):
            time_used = np.concatenate((time_used,time_used_base + 24*12* i))
        X=X[:,time_used.astype(np.int)]
        capacities = np.array(files_info['capacity'])
        capacities = capacities.astype('float32')
    elif dataset == 'ndbc':
        A, X = load_ndbc_data()
        print("A's shape is:",A.shape)# A's shape is: (93, 93)
        print("X's shape is:",X.shape)#X's shape is: (93, 8784)
    elif dataset == 'ushcn':
        A,X,Omissing = load_udata()
        X = X[:,:,:,0]
        X = X.reshape(1218,120*12)
        X = X/100
    elif dataset == 'sedata':
        A, X = load_sedata()
        A = A.astype('float32')
        X = X.astype('float32')
    elif dataset == 'pems':
        A,X = load_pems_data()
    else:
        raise NotImplementedError('Please specify datasets from: metr, nrel, ushcn, sedata or pems')
    split_line1 = int(X.shape[1] * 0.7)
    training_set = X[:,:split_line1].transpose()
    test_set = X[:, split_line1:].transpose()       # split the training and test period
    rand = np.random.RandomState(0) # Fixed random output

    NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
    NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
    NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
    Adj_dist=adj_dist(NDBC_lat,NDBC_long)
    j=5#J-th colum, start from 5 as the first feature.
    X_raw_0=X_raw[:,j,:]#GET the first feature
    print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    print("Incomplete data number in each column：", missing_counts_per_column)

    #Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    print("The column numbers with missing values are：", missing_columns)

    # Get new data after deletion.
    # Delete those columns (Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    print(result.shape)#(8784, 93)

    #Determine group num here
    group_num=result.shape[1]-N_u

    # Initialize MI_store
    MI_store = np.zeros((result.shape[1], result.shape[1]))

    # Calculate mutual information
    for i in range(result.shape[1]):
        for j in range(result.shape[1]):
            MI_store[i, j] = calc_MI(result[:, i], result[:, j], 10)

    # Return column numbers and corresponding mutual information
    column_numbers = []
    MI_values = []

    for i in range(MI_store.shape[0]):
        row = MI_store[i, :]
        row_index = np.arange(MI_store.shape[0])
        row_index = np.delete(row_index, i)  # Exclude the current row's index
        sorted_indices = np.argsort(row[row_index])
        max_indices = row_index[sorted_indices[-group_num:][::-1]]  # Get largest column_num
        column_numbers.append(max_indices)
        MI_values.append([row[j] for j in max_indices])

    # Output results
    for i, (columns, values) in enumerate(zip(column_numbers, MI_values)):
        print(f"Row {i} - Column Numbers: {columns}, MI Values: {values}")


    know_set=np.array(column_numbers)[random.randint(0, X.shape[0]-1),:]
    know_set=set(know_set)
    full_set = set(range(0,X.shape[0]))
    unknow_set = full_set - know_set
    training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
    A_s = A[:, list(know_set)][list(know_set), :]      # get the observed adjacent matrix from the full adjacent matrix,
                                                    # the adjacent matrix are based on pairwise distance,
                                                    # so we need not to construct it for each batch, we just use index to find the dynamic adjacent matrix
    return A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity

In [9]:

# from utils import load_data_MI

import torch
import numpy as np
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import *
import random
import pandas as pd
from basic_structure import IGNNK
from basic_process import *
import argparse
import sys
import os
import time
def parse_args(args):
    '''Parse training options user can specify in command line.
    Specify hyper parameters here
    Returns
    -------
    argparse.Namespace
        the output parser object
    '''
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description="Parse argument used when training IGNNK model.",
        epilog="python IGNNK_train.py DATASET, for example: python IGNNK_train.py 'metr' ")
    # Requird input parametrs
    parser.add_argument(
        'dataset',type=str,default='metr',
        help = 'Name of the datasets, select from metr,ndbc, nrel, ushcn, sedata or pems'
    )
    # optional input parameters
    parser.add_argument(
        '--n_o',type=int,default=20,
        help='sampled space dimension'
    )
    parser.add_argument(
        '--h',type=int,default=24,
        help='sampled time dimension'
    )
    parser.add_argument(
        '--z',type=int,default=100,
        help='hidden dimension for graph convolution'
    )
    parser.add_argument(
        '--K',type=int,default=1,
        help='If using diffusion convolution, the actual diffusion convolution step is K+1'
    )
    parser.add_argument(
        '--n_m',type=int,default=10,
        help='number of mask node during training'
    )
    parser.add_argument(
        '--n_u',type=int,default=50,
        help='target locations, n_u locations will be deleted from the training data'
    )
    parser.add_argument(
        '--max_iter',type=int,default=100,
        help='max training episode'
    )
    parser.add_argument(
        '--learning_rate',type=float,default=0.0001,
        help='the learning_rate for Adam optimizer'
    )
    parser.add_argument(
        '--E_maxvalue',type=int,default=80,
        help='the max value from experience'
    )
    parser.add_argument(
        '--batch_size',type=int,default=4,
        help='Batch size'
    )
    parser.add_argument(
        '--to_plot',type=bool,default=True,
        help='Whether to plot the RMSE training result'
    )
    return parser.parse_known_args(args)[0]

if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

usage: ipykernel_launcher.py [-h] [--n_o N_O] [--h H] [--z Z] [--K K]
                             [--n_m N_M] [--n_u N_U] [--max_iter MAX_ITER]
                             [--learning_rate LEARNING_RATE]
                             [--E_maxvalue E_MAXVALUE]
                             [--batch_size BATCH_SIZE] [--to_plot TO_PLOT]
                             dataset
ipykernel_launcher.py: error: the following arguments are required: dataset


SystemExit: 2

d:\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
def test_error_new(STmodel, unknow_set, test_data, A_s, Missing0):
    """
    :param STmodel: The graph neural networks
    :unknow_set: The unknow locations for spatial prediction
    :test_data: The true value test_data of shape (test_num_timesteps, num_nodes)
    :A_s: The full adjacent matrix
    :Missing0: True: 0 in original datasets means missing data
    :return: NAE, MAPE and RMSE
    """
    unknow_set = set(unknow_set)
    time_dim = STmodel.time_dimension
    test_omask = np.ones(test_data.shape)
    if Missing0 == True:
        test_omask[test_data == 0] = 0
    test_inputs = (test_data * test_omask).astype('float32')
    test_inputs_s = test_inputs
    missing_index = np.ones(np.shape(test_data))
    missing_index[:, list(unknow_set)] = 0
    missing_index_s = missing_index
    o = np.zeros([test_data.shape[0]//time_dim*time_dim, test_inputs_s.shape[1]]) #Separate the test data into several h period
    for i in range(0, test_data.shape[0]//time_dim*time_dim, time_dim):
        inputs = test_inputs_s[i:i+time_dim, :]
        missing_inputs = missing_index_s[i:i+time_dim, :]
        T_inputs = inputs*missing_inputs
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        T_inputs = T_inputs/E_maxvalue
        T_inputs = np.expand_dims(T_inputs, axis = 0)
        T_inputs = torch.from_numpy(T_inputs.astype('float32'))
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_s).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_s.T).T).astype('float32'))
        imputation = STmodel(T_inputs, A_q, A_h)
        imputation = imputation.data.numpy()
        o[i:i+time_dim, :] = imputation[0, :, :]
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    if dataset == 'NREL':
        dataset=dataset
    else:
        o = o*E_maxvalue
    truth = test_inputs_s[0:test_set.shape[0]//time_dim*time_dim]
    o[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1] = truth[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1]
    test_mask =  1 - missing_index_s[0:test_set.shape[0]//time_dim*time_dim]
    if Missing0 == True:
        test_mask[truth == 0] = 0
        o[truth == 0] = 0
    MAE = np.sum(np.abs(o - truth))/np.sum( test_mask)
    RMSE = np.sqrt(np.sum((o - truth)*(o - truth))/np.sum( test_mask) )
    MAPE = np.sum(np.abs(o - truth)/(truth + 1e-5))/np.sum( test_mask)
    return MAE, RMSE, MAPE

In [13]:
# load dataset
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity = load_data_MI('ndbc')

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'd:/Spatial_interpolation/IGNNK/NDBC/all_stations'

In [76]:

def test_error(STmodel, unknow_set, test_data, A_s, Missing0):
    """
    :param STmodel: The graph neural networks
    :unknow_set: The unknow locations for spatial prediction
    :test_data: The true value test_data of shape (test_num_timesteps, num_nodes)
    :A_s: The full adjacent matrix
    :Missing0: True: 0 in original datasets means missing data
    :return: NAE, MAPE and RMSE
    """
    unknow_set = set(unknow_set)
    time_dim = STmodel.time_dimension
    test_omask = np.ones(test_data.shape)
    if Missing0 == True:
        test_omask[test_data == 0] = 0
    test_inputs = (test_data * test_omask).astype('float32')
    test_inputs_s = test_inputs
    missing_index = np.ones(np.shape(test_data))
    missing_index[:, list(unknow_set)] = 0
    missing_index_s = missing_index
    o = np.zeros([test_data.shape[0]//time_dim*time_dim, test_inputs_s.shape[1]]) #Separate the test data into several h period
    for i in range(0, test_data.shape[0]//time_dim*time_dim, time_dim):
        inputs = test_inputs_s[i:i+time_dim, :]
        missing_inputs = missing_index_s[i:i+time_dim, :]
        T_inputs = inputs*missing_inputs
        T_inputs = T_inputs/E_maxvalue
        T_inputs = np.expand_dims(T_inputs, axis = 0)
        T_inputs = torch.from_numpy(T_inputs.astype('float32'))
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_s).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_s.T).T).astype('float32'))
        imputation = STmodel(T_inputs, A_q, A_h)
        imputation = imputation.data.numpy()
        o[i:i+time_dim, :] = imputation[0, :, :]
    if dataset == 'NREL':
        o = o*capacities[None,:]
    else:
        o = o*E_maxvalue
    truth = test_inputs_s[0:test_set.shape[0]//time_dim*time_dim]
    o[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1] = truth[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1]
    test_mask =  1 - missing_index_s[0:test_set.shape[0]//time_dim*time_dim]
    if Missing0 == True:
        test_mask[truth == 0] = 0
        o[truth == 0] = 0
    MAE = np.sum(np.abs(o - truth))/np.sum( test_mask)
    RMSE = np.sqrt(np.sum((o - truth)*(o - truth))/np.sum( test_mask) )
    MAPE = np.sum(np.abs(o - truth)/(truth + 1e-5))/np.sum( test_mask)
    return MAE, RMSE, MAPE

In [64]:
#torch.save(STmodel.state_dict(), '/content/Spatial-interpolation/model/IGNNK_{:}_{:}iter_{:}.pth'.format(dataset,max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
torch.save(STmodel.state_dict(), '/home/xren451/rxb/phd/Spatial_interpolation/XBSPA/ModIGNNK/IGNNK_{:}iter_{:}.pth'.format(max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
STmodel.state_dict()

OrderedDict([('GNN1.Theta1',
              tensor([[-0.0565, -0.0106,  0.0122,  ..., -0.0457, -0.0026, -0.0830],
                      [ 0.0296, -0.0353, -0.0647,  ...,  0.0538,  0.0159, -0.0188],
                      [ 0.0064,  0.0306, -0.0617,  ...,  0.0932, -0.0761, -0.0050],
                      ...,
                      [ 0.0333, -0.0776, -0.0641,  ...,  0.0298,  0.0085,  0.0693],
                      [-0.0389,  0.0677, -0.0106,  ...,  0.0133, -0.0053, -0.0038],
                      [-0.0375,  0.0726, -0.0529,  ..., -0.0637, -0.0726, -0.0040]])),
             ('GNN1.bias',
              tensor([ 0.0516, -0.0006, -0.0130,  0.0840,  0.0660,  0.0443,  0.0540, -0.0252,
                      -0.0925, -0.0581, -0.0634,  0.0417,  0.0515,  0.0052,  0.0355, -0.0728,
                       0.0627, -0.0744, -0.0446, -0.0263, -0.0756, -0.0776, -0.0634, -0.0239,
                       0.0590,  0.0439, -0.0400,  0.0145,  0.0888,  0.0073,  0.0227, -0.0540,
                      -0.0872, -0.

Inference

In [77]:
 # Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        MAE_t, RMSE_t, MAPE_t = test_error(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)


KeyboardInterrupt: 

In [66]:
#Adj---Determined by tSNE
#Raw data and raw adjacency matrix without mapminmax
X_raw
# X_raw=normalize_3d_array(X_raw)

for j in range(5,X_raw.shape[1]-1):
    NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
    NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
    NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
    Adj_dist=adj_dist(NDBC_lat,NDBC_long)
    X_raw_0=X_raw[:,j,:]#GET the first feature
    #print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    # print("Incomplete data number in each column：", missing_counts_per_column)

    #Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    #print("The column numbers with missing values are：", missing_columns)

    #Get new data after deletion.
    #Delete those columns(Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    #Mean results---for SNE
    mean_result_axis_1=np.mean(result,axis=0)
    mean_result_axis_1=mean_result_axis_1.reshape(-1,1)

    #Get new adjacency matrix after deletion.
    NDBC_long=NDBC_long.transpose()
    NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the longitude after deletion.
    NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
    print(NDBC_long_aft.shape)
    Temp_long=NDBC_long_aft.values

    NDBC_lat=NDBC_lat.transpose()
    NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the latitude after deletion.
    NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
    print(NDBC_lat_aft.shape)
    Temp_lat=NDBC_lat_aft.values

    merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
    tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
    array_aft = tsne.fit_transform(merged_array)

    #GEt the new ADJ matrix.
    Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
    capacities=np.ones((Adj_dist.shape[0], 1))
    capacities=capacities.flatten()
    split_line1 = int(result.shape[0] * 0.7)
    training_set = result[:split_line1, :]
    test_set = result[split_line1:, :]
    rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
    unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
    unknow_set = set(unknow_set)
    full_set = set(range(0,result.shape[1]))
    know_set = full_set - unknow_set
    training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
    A_s = Adj_dist[:, list(know_set)][list(know_set), :]
    MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
    print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

(1, 93)
(1, 93)
Best model in the 5 -th column is 0.4111344866707037 0.5056336246634306 -1.9637222193877926
(1, 96)
(1, 96)
Best model in the 6 -th column is 0.1569200881892884 0.18950212784464243 -2.315481521599812
(1, 94)
(1, 94)
Best model in the 7 -th column is 0.123197231781337 0.14963190894022255 -2.1610636131006586
(1, 72)
(1, 72)
Best model in the 8 -th column is 0.08809067083306635 0.1155820568433118 -1.2654039093997298
(1, 72)
(1, 72)
Best model in the 9 -th column is 0.2594348505503884 0.30182456286921516 -2.9660828065124076
(1, 65)
(1, 65)
Best model in the 10 -th column is 0.15511802607321726 0.17486788771174738 -3.911176053897554
(1, 40)
(1, 40)
Best model in the 11 -th column is 0.4654388023399339 0.5453110535030439 -2.6119704583039076
(1, 98)
(1, 98)
Best model in the 12 -th column is 0.05123041494218167 0.05410537383474029 -18.276946409701072
(1, 99)
(1, 99)
Best model in the 13 -th column is 0.6036140219756547 0.653074709208811 -38.21032856046987
(1, 77)
(1, 77)
Best 

Experiment 3.2: IGNNK-dis-MIsubgrh-3DGNN (Format:Metd-Adj-a node can constrct multiple subgraphs-3DGNN)

In [67]:
if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

In [68]:
# load dataset
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity = load_data_MI('ndbc')


(1, 93)
A's shape is: (93, 93)
X's shape is: (93, 8784)
(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175

In [69]:
from __future__ import division
import torch
import numpy as np
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import *
import random
import pandas as pd
from basic_structure import IGNNK
from basic_process import *
import argparse
import sys
import os
import time
import pdb
 # Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)

0 6.222990159801959 8.272169577972118 1755.1133693492645
1 2.060493961992602 2.946200733114902 549.7547794392173
2 1.1440364542468404 1.887073392540342 310.14670913027436
3 1.1028264199721072 1.8052983774290348 308.51296281390034
4 1.0545692468122156 1.7341308708072314 296.50541036719596
5 1.0070209801973136 1.6639515281850525 285.08238206469935
6 0.9677448835034214 1.5946733000818833 281.26018177665213
7 0.929201220541394 1.5258264683964489 271.9203028945845
8 0.8886728879154054 1.4606936961212476 262.3444857562488
9 0.855847464265839 1.3947589348047607 255.76327521105458
10 0.8152955931674924 1.331150455961094 245.1926086566716
11 0.7726473182688388 1.2537513610082385 236.67745276895036
12 0.7402393000402694 1.1903772049563395 231.2488214528439
13 0.7030370738391963 1.1301354863599085 218.68800561261148
14 0.6705016311119073 1.0736690439454823 210.50895312716557
15 0.644319384160054 1.0222397677814317 209.21220430259694
16 0.6166206365812015 0.9754995411777607 202.38673071795083
17 0

In [70]:
#torch.save(STmodel.state_dict(), '/content/Spatial-interpolation/model/IGNNK_{:}_{:}iter_{:}.pth'.format(dataset,max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
torch.save(STmodel.state_dict(), '/home/xren451/rxb/phd/Spatial_interpolation/XBSPA/ModIGNNK/model/IGNNK_{:}iter_{:}.pth'.format(max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
STmodel.state_dict()

OrderedDict([('GNN1.Theta1',
              tensor([[ 0.0707, -0.1025, -0.0382,  ...,  0.0399, -0.0018, -0.0219],
                      [-0.1037,  0.0218, -0.0364,  ...,  0.0359, -0.0929,  0.0590],
                      [ 0.0215,  0.0093, -0.0310,  ..., -0.0242,  0.0077, -0.0092],
                      ...,
                      [ 0.0019,  0.0768, -0.0372,  ...,  0.0330, -0.0324, -0.0109],
                      [ 0.0276, -0.0166,  0.0713,  ...,  0.0801, -0.0640,  0.0071],
                      [-0.0162, -0.1002, -0.0541,  ..., -0.0625, -0.0653, -0.0825]])),
             ('GNN1.bias',
              tensor([ 0.0497,  0.1095,  0.0891,  0.0449, -0.0007,  0.0672,  0.0725, -0.0668,
                      -0.0887,  0.0034, -0.0285,  0.0225,  0.0504, -0.0971,  0.0891,  0.0455,
                       0.0599,  0.0421,  0.0897, -0.0510,  0.0080,  0.0690, -0.0478, -0.0440,
                       0.0614, -0.0099,  0.0830, -0.0845,  0.0114,  0.0559,  0.0984,  0.0417,
                      -0.0144,  0.

In [71]:
#Subgraph generation module 1 --->Giving raw series, and generate a series of subgraph based on MI.
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
j=5#J-th colum, start from 5 as the first feature.
X_raw_0=X_raw[:,j,:]#GET the first feature
print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
print("Incomplete data number in each column：", missing_counts_per_column)

#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

# Print results
print("The column numbers with missing values are：", missing_columns)

# Get new data after deletion.
# Delete those columns (Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
print(result.shape)#(8784, 93)

#Determine group num here
group_num=result.shape[1]-N_u

# Initialize MI_store
MI_store = np.zeros((result.shape[1], result.shape[1]))

# Calculate mutual information
for i in range(result.shape[1]):
    for j in range(result.shape[1]):
        MI_store[i, j] = calc_MI(result[:, i], result[:, j], 10)

# Return column numbers and corresponding mutual information
column_numbers = []
MI_values = []

for i in range(MI_store.shape[0]):
    row = MI_store[i, :]
    row_index = np.arange(MI_store.shape[0])
    row_index = np.delete(row_index, i)  # Exclude the current row's index
    sorted_indices = np.argsort(row[row_index])
    max_indices = row_index[sorted_indices[-group_num:][::-1]]  # Get largest column_num
    column_numbers.append(max_indices)
    MI_values.append([row[j] for j in max_indices])

# Output results
for i, (columns, values) in enumerate(zip(column_numbers, MI_values)):
    print(f"Row {i} - Column Numbers: {columns}, MI Values: {values}")

(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175463517, 0.37428008133733837, 0.37263024544485734, 0.3321

In [72]:
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity,unknow_set_2D,know_set_2D,training_set_s_3D = load_data_MI_3D('ndbc')

(1, 93)
A's shape is: (93, 93)
X's shape is: (93, 8784)
(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175

In [73]:
training_set_s_3D.shape

(93, 6148, 43)

In [74]:
unknow_set_2D.shape

AttributeError: 'dict' object has no attribute 'shape'

In [80]:
training_set_s

{0: {0,
  6,
  7,
  9,
  14,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  36,
  37,
  38,
  39,
  40,
  45,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  57,
  58,
  59,
  60,
  62,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  75,
  76,
  80,
  81,
  82,
  84,
  86,
  88,
  89},
 1: {1,
  6,
  7,
  9,
  14,
  23,
  25,
  26,
  27,
  29,
  32,
  37,
  38,
  39,
  40,
  41,
  45,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  57,
  58,
  59,
  60,
  62,
  63,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  75,
  76,
  80,
  81,
  82,
  84,
  86,
  88,
  89},
 2: {2,
  6,
  7,
  8,
  9,
  14,
  16,
  18,
  22,
  23,
  24,
  25,
  26,
  27,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  62,
  63,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  75,
  76,
  80,
  81,
  82,
  84,
  88,
  90,
  92},
 3: {3,
  6,
  7,
  8,
  9,
  13,
  14,
  16,
  18,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  45,
 

In [ ]:
## test_err 函数： input：(STmodel, unknow_set, test_data, A_s, Missing0)，output:mae,mse,mape.
